In [1]:
%load_ext autoreload
%autoreload 2

import torch
from config import conf
from torch.utils.data import DataLoader
from torch.optim import Adam
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering
from tqdm import tqdm
from dataset import SquadDataset

In [2]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
dataset = SquadDataset.from_json(conf['DATASET_FILE'], tokenizer)
train_dataset, val_dataset = dataset.train_val_split(conf['TRAIN_RATIO'])

model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [3]:
def compute_accuracy(pred: torch.Tensor, true: torch.Tensor) -> float:
    assert len(pred) == len(true)
    return ((start_pred == start_true).sum() / len(start_pred)).item()

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

opt = Adam(model.parameters(), lr=5e-5)
train_loader = DataLoader(train_dataset, batch_size=conf['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=conf['BATCH_SIZE'])

for epoch in range(conf['N_EPOCHS']):
    # ================================ TRAINING ================================
    model.train()

    train_losses = []
    train_accs = []

    train_iter = tqdm(train_loader)
    train_iter.set_description(f'Epoch {epoch}')
    
    for train_batch in train_iter:
        input_ids = train_batch['input_ids'].to(device)
        attention_mask = train_batch['attention_mask'].to(device)
        start_true = train_batch['start_positions'].to(device)
        end_true = train_batch['end_positions'].to(device)

        outputs = model(input_ids, 
                        attention_mask=attention_mask,
                        start_positions=start_true,
                        end_positions=end_true)
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)

        loss = outputs['loss']
        start_acc = compute_accuracy(start_pred, start_true)
        end_acc = compute_accuracy(end_pred, end_true)

        train_iter.set_postfix(loss=loss.item(), 
                               acc=(start_acc + end_acc) / 2)
        
        train_losses.append(loss.item())
        train_accs.append(start_acc)
        train_accs.append(end_acc)
        
        loss.backward()
        
        opt.step()
        opt.zero_grad()

    # =============================== VALIDATION ===============================
    model.eval()
    
    val_losses = []
    val_accs = []

    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch['input_ids'].to(device)
            attention_mask = val_batch['attention_mask'].to(device)
            start_true = val_batch['start_positions'].to(device)
            end_true = val_batch['end_positions'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            start_pred = torch.argmax(outputs['start_logits'], dim=1)
            end_pred = torch.argmax(outputs['end_logits'], dim=1)

            val_losses.append(outputs['loss'])
            val_accs.append(compute_accuracy(start_pred, start_true))
            val_accs.append(compute_accuracy(end_pred, end_true))
    
    train_iter.set_postfix(loss=sum(train_losses) / len(train_losses),
                           acc=sum(train_accs) / len(train_accs),
                           val_loss=sum(val_losses) / len(val_losses),
                           val_acc=sum(val_accs) / len(val_accs))

Epoch 0:   0%|          | 0/2054 [00:00<?, ?it/s]


KeyError: 18258